# raxpy API Walkthrough

This notebook provides a high-level introduction to the use of raxpy to conduct and design experiments on Python functions. To get started, we import raxpy after installation. We also import Python's standard typing and dataclass capabilities to support expressive Python annotation.

In [1]:
from typing import Optional, Union, Annotated
from dataclasses import dataclass

import raxpy

raxpy introspects annotated Python functions to derive and execute experiments on the function. See below for the conversion of a non-annotated Python function, to a function with type-hints, and finally to an annotated function.

In [2]:
# simple function without type hints
def f1(x1, x2, x3):
    return x1 * x2 if x3 is None else x1 * x2 * x3

# simple function with type hints
def f2(x1: float, x2:int, x3:Optional[float]) -> float:
    return x1 * x2 if x3 is None else x1 * x2 * x3

# simple function with type hints and annotations
def f3(
    x1: Annotated[float, raxpy.Float(lb=0.0, ub=10.0)],
    x2: Annotated[float, raxpy.Float(lb=0.0, ub=2.0)],
    x3: Annotated[Optional[float], raxpy.Float(value_set=[0.0, 1.5, 3.0])]
) -> Annotated[float, raxpy.Float(tags=[raxpy.tags.MAXIMIZE])]:
    return x1 * x2 if x3 is None else x1 * x2 * x3 

To execute an experiment with an annotated Python function, we can simply execute the following code.

In [ ]:
inputs, outputs = raxpy.perform_experiment(f3, 10)

If we want to design an experiment but no execute it, we can simply execute the following code:

In [ ]:
doe = raxpy.design_experiment(f3, 10)

We can also decorate a function to ensure the function arguments are validated on every function call.

In [ ]:
# simple function with annotations and runtime validation
@raxpy.validate_at_runtime(check_outputs=False)
def f4(
    x1: Annotated[float, raxpy.Float(lb=0.0, ub=10.0)],
    x2: Annotated[int, raxpy.Float(lb=0.0, ub=2.0)],
    x3: Annotated[Optional[float], raxpy.Float(value_set=[0.0, 1.5, 3.0])]
) -> float:
    return x1 * x2 if x3 is None else x1 * x2 * x3

f4(3.14, 1, None) # no error
f4(3.14, 11, None) # runtime error given 11 value does not fall within range

The following code demonstrates the ability to annotate dataclasses attributes as well.

In [6]:
@dataclass
class HierarchicalFactorOne:
    x4: Annotated[float, raxpy.Float(lb=0.0, ub=1.0)]
    x5: Annotated[float, raxpy.Float(lb=0.0, ub=2.0)]

@dataclass
class HierarchicalFactorTwo:
    x6: Annotated[float, raxpy.Float(lb=0.0, ub=1.0)]
    x7: Annotated[float, raxpy.Float(lb=-1.0, ub=1.0)]


def f5(
    x1: Annotated[float, raxpy.Float(lb=0.0, ub=1.0)],
    x2: Annotated[Optional[float], raxpy.Float(lb=-1.0, ub=1.0)],
    x3: Union[HierarchicalFactorOne, HierarchicalFactorTwo],
):
    # placeholder for f5 logic
    return 1 


In [ ]:
f5_inputs, f5_outputs = raxpy.perform_experiment(f5, 25)

raxpy also supports manual specification of an experiment's input space.

In [8]:
simple_space = raxpy.spaces.InputSpace(
    dimensions=[
        raxpy.spaces.Float(id="x1", lb=0.0, ub=1.0, portion_null=0.0),
        raxpy.spaces.Float(id="x2", lb=0.0, ub=1.0, portion_null=0.0),
        raxpy.spaces.Float(id="x3", lb=0.0, ub=1.0, nullable=True, portion_null=0.1),
    ]
)
# more complex space specification
space = raxpy.spaces.InputSpace(
    dimensions=[
            raxpy.spaces.Float(id="x1", lb=0.0, ub=1.0, portion_null=0.0),
            raxpy.spaces.Float(
                id="x2",
                lb=0.0,
                ub=1.0,
                nullable=True,
                portion_null=1.0 / 3.0,
            ),
            raxpy.spaces.Composite(
                id="x3",
                nullable=True,
                portion_null=1.0 / 3.0,
                children=[
                    raxpy.spaces.Float(
                        id="x4",
                        lb=0.0,
                        ub=1.0,
                        portion_null=0.0,
                    ),
                    raxpy.spaces.Composite(
                        id="x5",
                        nullable=True,
                        portion_null=1.0 / 3.0,
                        children=[
                            raxpy.spaces.Float(
                                id="x6",
                                lb=0.0,
                                ub=1.0,
                                portion_null=0.0,
                            ),
                            raxpy.spaces.Float(
                                id="x7",
                                lb=0.0,
                                ub=1.0,
                                nullable=True,
                                portion_null=1.0 / 3.0,
                            ),
                        ],
                    ),
                ],
            ),
            raxpy.spaces.Composite(
                id="x8",
                nullable=True,
                portion_null=1.0 / 3.0,
                children=[
                    raxpy.spaces.Float(
                        id="x9",
                        lb=0.0,
                        ub=1.0,
                        portion_null=0.0,
                    ),
                    raxpy.spaces.Float(
                        id="x10",
                        lb=0.0,
                        ub=1.0,
                        nullable=True,
                        portion_null=1.0 / 3.0,
                    ),
                ],
            ),
    ]
)

raxpy uses a concept referred to a full-sub-spaces (FSS) to support designing and measursing experiment designs with optional and heirarcal dimensions. To compute these for a space we execute the following code.

In [ ]:
dim_sets = space.derive_full_subspaces()

raxpy supports computing numerous experiment design metrics related to designs' space filling properties.

In [12]:
from raxpy.does.doe import EncodingEnum

# Note that some metric compuations require the specification
# of a design encoding to use for measurement. While the 
# EncodingEnum.ZERO_ONE_NULL_ENCODING is highly suggested,
# some experiment design algorithms result in a decoded 
# (EncodingEnum.NONE) design. raxpy does not currently have
# logic to re-encode a design from a decoded design.

m_opt_coverage = raxpy.measure.compute_opt_coverage(doe)
m_min_interpoint_dist = raxpy.measure.compute_min_interpoint_dist(
    doe,
    encoding=EncodingEnum.NONE,
    p=2,
)
m_discrepancy = raxpy.measure.compute_star_discrepancy(doe,encoding=EncodingEnum.NONE)
m_avg_min_proj_dist = raxpy.measure.compute_average_dim_dist(doe)
m_max_pro = raxpy.measure.compute_max_pro(doe,encoding=EncodingEnum.NONE)